# Abcd vs abcD


### Modules

In [1]:
#import os
#import numpy as np
import sympy as sp
sp.init_printing(use_latex='mathjax')
#import pandas as pd
#import ast
import itertools as itr

#from multiprocessing import Pool, cpu_count
#from tqdm.notebook import tqdm

from snappy import *
S = twister.Surface('S_2_1')

## Symplectic representation

In [277]:
K = sp.MatrixSymbol('K',2,2) #([[0,-1],[0,0]])
L = sp.MatrixSymbol('L',2,2) #([[1,0],[1,1]])
I = sp.Identity(2)
O = sp.ZeroMatrix(2,2)
J = sp.MatrixSymbol('J',2,2) #([[0,1],[-1,0]])

Ma = sp.BlockMatrix([[L.inv(), O], [O, I]])
Mb = sp.BlockMatrix([[L.transpose(), K], [K, L.transpose()]])
Mc = sp.BlockMatrix([[I, O], [O, L.inv()]])
Md = sp.BlockMatrix([[I, O], [O, L.transpose()]])
Mf = sp.BlockMatrix([[L.transpose(), O], [O,I]])
#display(Ma, Mb, Mc, Md)

MJ = sp.BlockMatrix([[J, O], [O, J]])
#display(MJ)

### Functions

In [255]:
def ev(M):
    return M.subs([
        (L, sp.Matrix([[1,0],[1,1]])),
        (K, sp.Matrix([[0,-1],[0,0]])),
        (J, sp.Matrix([[0,1],[-1,0]])),
        (br, sp.Matrix([[0,0],[0,1]])),
        (tl, sp.Matrix([[1,0],[0,0]])),
    ]).as_explicit()
#evMd = ev(Md); display(evMd, evMd.det())

def is_Sp(M):
    if type(M) == sp.matrices.expressions.blockmatrix.BlockMatrix:
        return sp.block_collapse(ev(M).transpose()*ev(MJ)*ev(M)) == ev(MJ)
    else:
        return M.transpose()*ev(MJ)*M == ev(MJ)

for M in {Ma, Mb, Mc, Md}:
    if is_Sp(M):
        print(f"{M} is in Sp(4,Z)")

Matrix([
[L**(-1), 0],
[      0, I]]) is in Sp(4,Z)
Matrix([
[L.T,   K],
[  K, L.T]]) is in Sp(4,Z)
Matrix([
[I,       0],
[0, L**(-1)]]) is in Sp(4,Z)
Matrix([
[I,   0],
[0, L.T]]) is in Sp(4,Z)


### The symplectic images of Abcd and abcD

In [586]:
M_Abcd = sp.block_collapse((Ma.inv())*Mb*Mc*Md)
M1 = ev(M_Abcd)
display(M_Abcd, M1)
M_abcD = sp.block_collapse(Ma*Mb*Mc*(Md.inv()))
M2 = ev(M_abcD)
display(M_abcD, M2)

for M in {M1,M2}:
    print(f"is_Sp? --> {is_Sp(M)}")

⎡   T       -1  T⎤
⎢L⋅L   L⋅K⋅L  ⋅L ⎥
⎢                ⎥
⎢       T  -1  T ⎥
⎣ K    L ⋅L  ⋅L  ⎦

⎡1  1   1   0⎤
⎢            ⎥
⎢1  2   1   0⎥
⎢            ⎥
⎢0  -1  0   1⎥
⎢            ⎥
⎣0  0   -1  0⎦

⎡                      -1⎤
⎢ -1  T   -1    -1 ⎛ T⎞  ⎥
⎢L  ⋅L   L  ⋅K⋅L  ⋅⎝L ⎠  ⎥
⎢                        ⎥
⎢                    -1  ⎥
⎢          T  -1 ⎛ T⎞    ⎥
⎣  K      L ⋅L  ⋅⎝L ⎠    ⎦

⎡1   1   1   -2⎤
⎢              ⎥
⎢-1  0   -1  2 ⎥
⎢              ⎥
⎢0   -1  0   1 ⎥
⎢              ⎥
⎣0   0   -1  2 ⎦

is_Sp? --> True
is_Sp? --> True


### The charactoristic polynomial of M1 (M2)

In [5]:
cp_M1 = M1.charpoly().as_expr()
cp_M2 = M2.charpoly().as_expr()

print(f"cp_M1 == cp_M2 ?   --> {cp_M1==cp_M2}")

display(cp_M1.factor())

cp_M1 == cp_M2 ?   --> True


 4      3      2          
λ  - 3⋅λ  + 3⋅λ  - 3⋅λ + 1

## Rational canonical form

### The rational form of M1 (M2)

In [6]:
C = sp.matrices.expressions.CompanionMatrix(M1.charpoly()).as_explicit()
display(C)
print(f"is_Sp? --> {is_Sp(C)}")

⎡0  0  0  -1⎤
⎢           ⎥
⎢1  0  0  3 ⎥
⎢           ⎥
⎢0  1  0  -3⎥
⎢           ⎥
⎣0  0  1  3 ⎦

is_Sp? --> False


### Functions

In [7]:
def transition_matrix(A,v):
    if type(v) == list:
        v = sp.Matrix(4,1,v)
    vecs = [(A**k)*v for k in range(5)]
    return sp.Matrix(4,4,lambda i,j: vecs[j][i])
def tm(A,v):
    return transition_matrix(A,v)

In [8]:
def get_matrix_connecting_M1_with_M2(v, w):
    P, Q = tm(M1,v), tm(M2,w)
    return (P, Q, P*(Q.inv()))

### Base symplectic matrix

In [9]:
v0, w0 = [0,-1,1,0], [1,0,0,0]
P, Q, S0 = get_matrix_connecting_M1_with_M2(v0,w0)
display(S0)
print(f"{(M2 == S0.inv()*M1*S0)=}, {is_Sp(S0)=}")

⎡0   0  0   1 ⎤
⎢             ⎥
⎢-1  0  -1  1 ⎥
⎢             ⎥
⎢1   0  0   0 ⎥
⎢             ⎥
⎣0   1  0   -1⎦

(M2 == S0.inv()*M1*S0)=True, is_Sp(S0)=True


### Generating other symplectic matrices from the base matrix

In [24]:
display(M1*S0)

v, w = [1,1,0,0], [0,1,1,1]
P, Q, X = get_matrix_connecting_M1_with_M2(v,w)
print(f"{(X == M1*S0)=}, {(M2 == X.inv()*M1*X)=}, {is_Sp(X)=}")

⎡0   0  -1  2 ⎤
⎢             ⎥
⎢-1  0  -2  3 ⎥
⎢             ⎥
⎢1   1  1   -2⎥
⎢             ⎥
⎣-1  0  0   0 ⎦

(X == M1*S0)=True, (M2 == X.inv()*M1*X)=True, is_Sp(X)=True


In [11]:
A = tm(M1,v)*(tm(M1,v0).inv())
v0, v = sp.Matrix(4,1,v0), sp.Matrix(4,1,v)
print(f"{(A*v0 == v) =}")

B = (S0.inv())*(M1**(-1))*A*S0
w0, w = sp.Matrix(4,1,w0), sp.Matrix(4,1,w)
print(f"{(B*w0 == w) =}")

(A*v0 == v) =True
(B*w0 == w) =True


In [12]:
for k in range(-5,5):
    P, Q, X = get_matrix_connecting_M1_with_M2((M1**k)*v0, w0)
    display(X)
    print(f"{k=}, {is_Sp(X)=}, {(M2 == (X.inv())*M1*X)=} \n--------------")

⎡26   14   20   3 ⎤
⎢                 ⎥
⎢-23  -12  -17  -3⎥
⎢                 ⎥
⎢ 9    4    6   2 ⎥
⎢                 ⎥
⎣-18  -9   -14  -3⎦

k=-5, is_Sp(X)=True, (M2 == (X.inv())*M1*X)=True 
--------------


⎡12   6   9   2 ⎤
⎢               ⎥
⎢-11  -6  -8  -1⎥
⎢               ⎥
⎢ 5   3   3   0 ⎥
⎢               ⎥
⎣-9   -4  -6  -2⎦

k=-4, is_Sp(X)=True, (M2 == (X.inv())*M1*X)=True 
--------------


⎡6   3   4   1 ⎤
⎢              ⎥
⎢-5  -3  -4  0 ⎥
⎢              ⎥
⎢2   2   2   -1⎥
⎢              ⎥
⎣-5  -3  -3  0 ⎦

...

k=2, is_Sp(X)=True, (M2 == (X.inv())*M1*X)=True 
--------------


⎡-1  2   -4  6 ⎤
⎢              ⎥
⎢-2  3   -8  12⎥
⎢              ⎥
⎢0   -2  3   -4⎥
⎢              ⎥
⎣0   0   -2  3 ⎦

k=3, is_Sp(X)=True, (M2 == (X.inv())*M1*X)=True 
--------------


⎡-3  3   -9   14⎤
⎢               ⎥
⎢-5  6   -17  26⎥
⎢               ⎥
⎢2   -3   6   -9⎥
⎢               ⎥
⎣0   2   -3   4 ⎦

k=4, is_Sp(X)=True, (M2 == (X.inv())*M1*X)=True 
--------------


In [13]:
v0, v = sp.Matrix(4,1,v0), sp.Matrix(4,1,[1, -1, 0, -1])
A = tm(M1,v)*(tm(M1,v0).inv())
print(f"{(A*v0 == v) =}")

w0, w = sp.Matrix(4,1,w0), sp.Matrix(4,1,[1, -1, 0, -1])

for k in range(-20, 20):
    B = (S0.inv())*(M1**k)*A*S0
    if B*w0 == w:
        print(k)

(A*v0 == v) =True
2


In [14]:
P, Q, X = get_matrix_connecting_M1_with_M2(v,w)
display(X)
print(f"{is_Sp(X)=}")

⎡3   2   2   0⎤
⎢             ⎥
⎢-2  -1  -2  0⎥
⎢             ⎥
⎢0   0   1   0⎥
⎢             ⎥
⎣-2  -2  -2  1⎦

is_Sp(X)=True


In [152]:
S1 = sp.Matrix([[0,1,0,-1],[-1,-1,0,0],[0,0,0,1],[-1,-1,-1,0]])
display(S1)

Q = (S1.inv())*(S0.inv())
print(Q*M1 == M1*Q)

#display(Q, M1**(-1))

print(S1.inv() == (-1)*M1.inv()*S0)

⎡0   1   0   -1⎤
⎢              ⎥
⎢-1  -1  0   0 ⎥
⎢              ⎥
⎢0   0   0   1 ⎥
⎢              ⎥
⎣-1  -1  -1  0 ⎦

True
True


# Lifting to a mapping class

## Generators of $Sp(4, \mathbb{Z})$

In [258]:
I4 = sp.BlockMatrix([[I, O], [O, I]])

A1 = sp.BlockMatrix([[L.transpose(), O], [O, I]])
A2 = sp.BlockMatrix([[I, O], [O, L.transpose()]])
display(A1, A2)
B1 = sp.BlockMatrix([[L, O], [O, I]])
B2 = sp.BlockMatrix([[I, O], [O, L]])
display(B1, B2)
C = sp.BlockMatrix([[I, -K], [-K, I]])
D = sp.BlockMatrix([[I, -K.transpose()], [-K.transpose(), I]])
display(C, D)
br, tl = sp.MatrixSymbol('br', 2,2), sp.MatrixSymbol('tl', 2,2)
E12 = sp.BlockMatrix([[I, -br], [tl, I]])
E21 = sp.BlockMatrix([[I, tl], [-br, I]])
display(E12, E21)

⎡ T   ⎤
⎢L   𝟘⎥
⎢     ⎥
⎣𝟘   𝕀⎦

⎡𝕀  𝟘 ⎤
⎢     ⎥
⎢    T⎥
⎣𝟘  L ⎦

⎡L  𝟘⎤
⎢    ⎥
⎣𝟘  𝕀⎦

⎡𝕀  𝟘⎤
⎢    ⎥
⎣𝟘  L⎦

⎡𝕀   -K⎤
⎢      ⎥
⎣-K  𝕀 ⎦

⎡       T⎤
⎢ 𝕀   -K ⎥
⎢        ⎥
⎢  T     ⎥
⎣-K    𝕀 ⎦

⎡𝕀   -br⎤
⎢       ⎥
⎣tl   𝕀 ⎦

⎡ 𝕀   tl⎤
⎢       ⎥
⎣-br  𝕀 ⎦

## Make basic matrices into products of $\{M_a, \dots, M_f\}$

### A, B, and C

In [608]:
[ev(A1) == ev(Mf), \
 ev(A2) == ev(Md), \
 #-----
 ev(B1) == ev(Ma.inv()), \
 ev(B2) == ev(Mc.inv()), \
 #-----
 ev(C) == ev(Mf*(Mb.inv())*Md)]

[True, True, True, True, True]

### D and X

In [609]:
ev(D) == ev(Ma*(Mb.transpose())*Mc).inv()

True

In [93]:
X = ev(Ma*(Mb.transpose())*Mc).transpose().inv()
display(X)

⎡1  0  0  1⎤
⎢          ⎥
⎢0  1  0  0⎥
⎢          ⎥
⎢0  1  1  0⎥
⎢          ⎥
⎣0  0  0  1⎦

### E

#### Trial 01

In [435]:
F = ev(Mb.transpose())*ev(Ma)
N = (F.inv())*ev(Mf.inv())*F
        
ev(E21), N*ev(Mc)*ev(Mf)

⎛⎡1  0   1  0⎤  ⎡1  0   1  0⎤⎞
⎜⎢           ⎥  ⎢           ⎥⎟
⎜⎢0  1   0  0⎥  ⎢0  1   0  0⎥⎟
⎜⎢           ⎥, ⎢           ⎥⎟
⎜⎢0  0   1  0⎥  ⎢0  0   1  0⎥⎟
⎜⎢           ⎥  ⎢           ⎥⎟
⎝⎣0  -1  0  1⎦  ⎣0  -1  0  1⎦⎠

In [391]:
G = ev(Mf.inv())*ev(Mb)
H = (G)*ev(Ma)*(G.inv())
        
ev(E12), ev(Ma.inv())*ev(Md.inv())*H

⎛⎡1  0  0  0 ⎤  ⎡1  0  0  0 ⎤⎞
⎜⎢           ⎥  ⎢           ⎥⎟
⎜⎢0  1  0  -1⎥  ⎢0  1  0  -1⎥⎟
⎜⎢           ⎥, ⎢           ⎥⎟
⎜⎢1  0  1  0 ⎥  ⎢1  0  1  0 ⎥⎟
⎜⎢           ⎥  ⎢           ⎥⎟
⎝⎣0  0  0  1 ⎦  ⎣0  0  0  1 ⎦⎠

##### Mb.transpose()

In [392]:
ev(Mb.transpose()), ev(E12*B2*(E12.inv()))  #, ev((MJ.inv())*(Mb.inv())*(MJ))

⎛⎡1   0  0   0⎤  ⎡1   0  0   0⎤⎞
⎜⎢            ⎥  ⎢            ⎥⎟
⎜⎢1   1  -1  0⎥  ⎢1   1  -1  0⎥⎟
⎜⎢            ⎥, ⎢            ⎥⎟
⎜⎢0   0  1   0⎥  ⎢0   0  1   0⎥⎟
⎜⎢            ⎥  ⎢            ⎥⎟
⎝⎣-1  0  1   1⎦  ⎣-1  0  1   1⎦⎠

In [421]:
ev((Md*Ma).inv())*H * ev(Mc.inv()) * (H.inv())*ev(Md*Ma)

⎡1   0  0   0⎤
⎢            ⎥
⎢1   1  -1  0⎥
⎢            ⎥
⎢0   0  1   0⎥
⎢            ⎥
⎣-1  0  1   1⎦

In [406]:
#(ev(Ma.inv())*ev(G)*ev(Ma)*ev(G.inv()))*ev(Md.inv())*ev(Mc.inv())*ev(Md)*(H.inv())*ev(Ma)
#(ev(Ma.inv())*ev(G)*ev(Ma)*ev(G.inv()))*ev(Md.inv())*ev(Mc.inv())*ev(Md)*(H.inv())*ev(Ma)
ev(Ma.inv()*H)*ev(Md.inv())*ev(Mc.inv())*ev(Md)*ev(H.inv()*Ma)

⎡1   0  0   0⎤
⎢            ⎥
⎢1   1  -1  0⎥
⎢            ⎥
⎢0   0  1   0⎥
⎢            ⎥
⎣-1  0  1   1⎦

In [407]:
def cm(X,Y):
    return X*Y*(X.inv())*(Y.inv())

In [418]:
CC = ev(cm(Mf.inv()*Mb, Ma.inv()))
((CC*ev(Md)).inv()) * ev(Mc.inv()) * (CC*ev(Md))

⎡1   0  0   0⎤
⎢            ⎥
⎢1   1  -1  0⎥
⎢            ⎥
⎢0   0  1   0⎥
⎢            ⎥
⎣-1  0  1   1⎦

In [441]:
CC = ev(cm(Md.inv()*Mb, Mc.inv()))
#F = ev(cm((CC*Mf).inv(), Ma.inv()))  #( = ev(Mb.transpose())*ev(Ma))
#N = (F.inv())*ev(Mf.inv())*F
        
ev(E21), ev(Mc)*ev(cm(cm(Ma.inv(), (CC*Mf).inv()), Mf.inv()))

⎛⎡1  0   1  0⎤  ⎡1  0   1  0⎤⎞
⎜⎢           ⎥  ⎢           ⎥⎟
⎜⎢0  1   0  0⎥  ⎢0  1   0  0⎥⎟
⎜⎢           ⎥, ⎢           ⎥⎟
⎜⎢0  0   1  0⎥  ⎢0  0   1  0⎥⎟
⎜⎢           ⎥  ⎢           ⎥⎟
⎝⎣0  -1  0  1⎦  ⎣0  -1  0  1⎦⎠

#### Trial 02

In [552]:
prd = {'left': [Md.inv(), Mc.inv(), Mf, Mb.inv(), Md, Mc], 'right': [Mb]}

Z = ev(E21)
for m in reversed(prd['left']):
    Z = ev(m)*Z
for m in prd['right']: 
    Z *= ev(m)
    
display(Z)

⎡1  0  0  0⎤
⎢          ⎥
⎢0  1  0  0⎥
⎢          ⎥
⎢0  0  1  0⎥
⎢          ⎥
⎣0  0  0  1⎦

In [555]:
ev(E21), ev((Mc.inv())*(Md.inv())*Mb*Mc*Md*(Mb.inv())*(Mf.inv()))

⎛⎡1  0   1  0⎤  ⎡1  0   1  0⎤⎞
⎜⎢           ⎥  ⎢           ⎥⎟
⎜⎢0  1   0  0⎥  ⎢0  1   0  0⎥⎟
⎜⎢           ⎥, ⎢           ⎥⎟
⎜⎢0  0   1  0⎥  ⎢0  0   1  0⎥⎟
⎜⎢           ⎥  ⎢           ⎥⎟
⎝⎣0  -1  0  1⎦  ⎣0  -1  0  1⎦⎠

## Decomposion of $S_0$

In [25]:
display(S0)

⎡0   0  0   1 ⎤
⎢             ⎥
⎢-1  0  -1  1 ⎥
⎢             ⎥
⎢1   0  0   0 ⎥
⎢             ⎥
⎣0   1  0   -1⎦

In [273]:
Z = ev((-1)*B1*(X.inv())*(A1.inv())*B1*(A1.inv())*E21)
S0, Z

⎛⎡0   0  0   1 ⎤  ⎡0   0  0   1 ⎤⎞
⎜⎢             ⎥  ⎢             ⎥⎟
⎜⎢-1  0  -1  1 ⎥  ⎢-1  0  -1  1 ⎥⎟
⎜⎢             ⎥, ⎢             ⎥⎟
⎜⎢1   0  0   0 ⎥  ⎢1   0  0   0 ⎥⎟
⎜⎢             ⎥  ⎢             ⎥⎟
⎝⎣0   1  0   -1⎦  ⎣0   1  0   -1⎦⎠

### Trial 01

In [334]:
Z = (-1)*ev((Md.inv()))*ev(Ma.inv())*ev(Mb)*ev(Mf.inv()**2)*ev(Ma.inv())*ev(Mf.inv())*ev(E21)
S0, Z

⎛⎡0   0  0   1 ⎤  ⎡0   0  0   1 ⎤⎞
⎜⎢             ⎥  ⎢             ⎥⎟
⎜⎢-1  0  -1  1 ⎥  ⎢-1  0  -1  1 ⎥⎟
⎜⎢             ⎥, ⎢             ⎥⎟
⎜⎢1   0  0   0 ⎥  ⎢1   0  0   0 ⎥⎟
⎜⎢             ⎥  ⎢             ⎥⎟
⎝⎣0   1  0   -1⎦  ⎣0   1  0   -1⎦⎠

In [348]:
Z = (Md.inv())*(Ma.inv())*Mb*(Mf.inv()**2)*(Ma.inv())*(Mf.inv())*N*Mf*Mc
S0, ev((-1)*Z)

⎛⎡0   0  0   1 ⎤  ⎡0   0  0   1 ⎤⎞
⎜⎢             ⎥  ⎢             ⎥⎟
⎜⎢-1  0  -1  1 ⎥  ⎢-1  0  -1  1 ⎥⎟
⎜⎢             ⎥, ⎢             ⎥⎟
⎜⎢1   0  0   0 ⎥  ⎢1   0  0   0 ⎥⎟
⎜⎢             ⎥  ⎢             ⎥⎟
⎝⎣0   1  0   -1⎦  ⎣0   1  0   -1⎦⎠

In [455]:
#marr = [Md.inv(), Ma.inv(), Mb, Mf.inv()**2, Ma.inv(), Mf.inv(), \
#        Mc, cm(cm(Ma.inv(), (CC*Mf).inv()), Mf.inv())]
marr = [Md.inv(), Ma.inv(), Mb*Mc, Mf.inv(), Ma.inv(), Mf.inv(), Ma.inv(), \
        cm(cm(Ma.inv(), ev((Mf.inv())*cm(Mc.inv(), Md.inv()*Mb))), Mf.inv())]
#marr = [Md.inv(), M1, Md.inv(), Mf.inv(), Ma.inv(), Mf.inv(), Ma.inv(), \
#        cm(cm(Ma.inv(), ev((Mf.inv())*cm(Mc.inv(), Md.inv()*Mb))), Mf.inv())]

Z = ev(I4)
for m in marr:
    Z *= ev(m)

S0, ev((-1)*Z)

⎛⎡0   0  0   1 ⎤  ⎡0   0  0   1 ⎤⎞
⎜⎢             ⎥  ⎢             ⎥⎟
⎜⎢-1  0  -1  1 ⎥  ⎢-1  0  -1  1 ⎥⎟
⎜⎢             ⎥, ⎢             ⎥⎟
⎜⎢1   0  0   0 ⎥  ⎢1   0  0   0 ⎥⎟
⎜⎢             ⎥  ⎢             ⎥⎟
⎝⎣0   1  0   -1⎦  ⎣0   1  0   -1⎦⎠

### Trial 02

In [560]:
prd = [Md.inv(), Ma.inv(), Mb, Mf.inv()**2, Ma.inv(), Mf.inv(), \
       Mc.inv(), Md.inv(), Mb, Mc, Md, Mb.inv(), Mf.inv()]

Z = (-1)*S0.inv()
for m in prd:
    Z *= ev(m)
    
display(Z)

⎡1  0  0  0⎤
⎢          ⎥
⎢0  1  0  0⎥
⎢          ⎥
⎢0  0  1  0⎥
⎢          ⎥
⎣0  0  0  1⎦

In [587]:
#prd = [M1.inv(), (Ma*Mb*Mc*Md).inv()]
prd = [(Ma.inv()*Mb*Mc*Md).inv(), (Ma*Mb*Mc*Md).inv()]
Z = S0
for m in prd:
    Z = ev(m)*Z

Z

⎡1  0  0  0⎤
⎢          ⎥
⎢0  1  0  0⎥
⎢          ⎥
⎢0  0  1  0⎥
⎢          ⎥
⎣0  0  0  1⎦

In [588]:
S0, ev((Ma.inv())*Mb*Mc*Md)*ev(Ma*Mb*Mc*Md)

⎛⎡0   0  0   1 ⎤  ⎡0   0  0   1 ⎤⎞
⎜⎢             ⎥  ⎢             ⎥⎟
⎜⎢-1  0  -1  1 ⎥  ⎢-1  0  -1  1 ⎥⎟
⎜⎢             ⎥, ⎢             ⎥⎟
⎜⎢1   0  0   0 ⎥  ⎢1   0  0   0 ⎥⎟
⎜⎢             ⎥  ⎢             ⎥⎟
⎝⎣0   1  0   -1⎦  ⎣0   1  0   -1⎦⎠

## Decomposion of S1

In [476]:
prd = {'left': [E21], 'right': [A1.inv()]}

Z = S1
for m in prd['left']: Z = ev(m)*Z
for m in prd['right']: Z *= ev(m)
    
display(Z)

⎡0   1  0   0⎤
⎢            ⎥
⎢-1  0  0   0⎥
⎢            ⎥
⎢0   0  0   1⎥
⎢            ⎥
⎣0   0  -1  0⎦

In [600]:
#ev(E21*S1*A1.inv()) == (-1)*ev(A1.inv())*ev(B1)*ev(A1.inv())*ev(A2.inv())*ev(B2)*ev(A2.inv())
(-1)*ev((E21.inv())*(A1.inv())*ev(B1)*(A1.inv())*(A2.inv())*ev(B2)*(A2.inv())*A1) == S1

True

In [602]:
#(-1)*ev((E21.inv())*((Md*Mc*Md).inv())*((Mf*Ma*Mf).inv())*Mf) == S1
#(-1)*ev((E21.inv())*((Md*Mc*Md).inv())*((Mf.inv())*(Ma.inv()))*(Mf.inv())*Mf) == S1
#(-1)*ev((E21.inv())*((Md*Mc*Md).inv())*((Ma*Mf).inv())) == S1
ev((Md*Mc*Md)*(Ma*Mf)*E21) == (-1)*(S1.inv())

True

In [601]:
#prd = [Md, Mc, Md, Ma, Mf, Mc.inv(), Md.inv(), Mb, Mc, Md, Mb.inv(), Mf.inv()]
#prd = [Md, Mc, Md, Ma, Mc.inv(), Md.inv(), Mb, Mc, Md, Mb.inv()]
#prd = [Mc, Md, Ma, Md.inv(), Mb, Mc, Md, Mb.inv()]
#prd = [Mc, Ma, Mb, Mc, Md, Mb.inv()]
prd = [Ma, Mb, Mc, Md]

Z = (-1)*S1
for m in prd:
    Z *= ev(m)

display(Z)

⎡1  0  0  0⎤
⎢          ⎥
⎢0  1  0  0⎥
⎢          ⎥
⎢0  0  1  0⎥
⎢          ⎥
⎣0  0  0  1⎦

In [592]:
(-1)*(S1.inv()), ev(Ma*Mb*Mc*Md)

⎛⎡1   1   1   0⎤  ⎡1   1   1   0⎤⎞
⎜⎢             ⎥  ⎢             ⎥⎟
⎜⎢-1  0   -1  0⎥  ⎢-1  0   -1  0⎥⎟
⎜⎢             ⎥, ⎢             ⎥⎟
⎜⎢0   -1  0   1⎥  ⎢0   -1  0   1⎥⎟
⎜⎢             ⎥  ⎢             ⎥⎟
⎝⎣0   0   -1  0⎦  ⎣0   0   -1  0⎦⎠

In [595]:
ev(M2), ev( ((Ma*Mb*Mc*Md).inv()) * M1 * (Ma*Mb*Mc*Md) )

⎛⎡1   1   1   -2⎤  ⎡1   1   1   -2⎤⎞
⎜⎢              ⎥  ⎢              ⎥⎟
⎜⎢-1  0   -1  2 ⎥  ⎢-1  0   -1  2 ⎥⎟
⎜⎢              ⎥, ⎢              ⎥⎟
⎜⎢0   -1  0   1 ⎥  ⎢0   -1  0   1 ⎥⎟
⎜⎢              ⎥  ⎢              ⎥⎟
⎝⎣0   0   -1  2 ⎦  ⎣0   0   -1  2 ⎦⎠

In [598]:
prd = [Md.inv(), Mc.inv(), Mb.inv(), Ma.inv(),\  # <-- (-1)*S1
       Ma.inv(), Mb, Mc, Md, \                   # <-- M1
       Ma, Mb, Mc, Md, \                         # <-- (-1)*S1.inv()
       Md, Mc.inv(), Mb.inv(), Ma.inv()]         # <-- M2.inv()

Z = I4
for m in prd:
    Z *= ev(m)

Z

⎡1  0  0  0⎤
⎢          ⎥
⎢0  1  0  0⎥
⎢          ⎥
⎢0  0  1  0⎥
⎢          ⎥
⎣0  0  0  1⎦

# Ajustment of the lift of $S_1$